In [2]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ff_energy.ffe.utils import read_from_pickle
import jax.numpy as jnp
from ff_energy.ffe.plot import plot_energy_MSE, plot_ff_fit
from ff_energy.ffe.structure import atom_key_pairs
from ff_energy.ffe.potential import LJ, akp_indx
from ff_energy.ffe.ff import FF
from ff_energy.ffe.ff_fit import LJ_bound, load_ff, fit_func, fit_repeat
from ff_energy.ffe.data import pairs_data
from ff_energy.ffe.utils import pickle_output, read_from_pickle, str2int




In [20]:
NDE = 10
NLJ = 10
sig_bound = (0.05, 2.5)
ep_bound = (0.001, 0.5)
LJ_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound))
DE_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound), 
            (1, 8), (6, 20))

In [21]:
#  example
# ffpkl = "ECOL_pbe0dz_FOE.ff.pkl"

In [22]:
# Parameters
ffpkl = "pc_pbe0dz_FOE.ff.pkl"


#  LJ fit


In [23]:
_c = next(read_from_pickle(f"ff/{ffpkl}"))
_c

FF: LJ water_cluster ELEC harmonic first_order_energy (jax_coloumb: True)

In [24]:
# _c.targets

In [25]:
fit_repeat(_c, 
           NLJ, 
           f"{ffpkl}_LJ",
           bounds = LJ_bound,
           loss="jax",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:174: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


In [26]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
1,15.818173,761,1306,0,True,Optimization terminated successfully.,"[0.7814906984824092, 2.4151860942057897, 0.001...","([[0.7814906984824092, 2.4151860942057897, 0.0..."
5,16.368975,668,1150,0,True,Optimization terminated successfully.,"[0.07199135183310364, 2.5, 0.25773598762695793...","([[0.07199135183310364, 2.5, 0.257735987626957..."
4,16.448004,830,1414,0,True,Optimization terminated successfully.,"[0.05000922558293672, 2.4963095272407907, 0.34...","([[0.05000922558293672, 2.4963095272407907, 0...."
8,16.622938,359,640,0,True,Optimization terminated successfully.,"[0.05, 2.445000380509922, 0.42639405360183547,...","([[0.05, 2.445000380509922, 0.4263940536018354..."
9,36.560619,180,335,0,True,Optimization terminated successfully.,"[0.7790504630173785, 1.8205451552485727, 0.001...","([[0.7790504630173785, 1.8205451552485727, 0.0..."
3,80.613953,406,738,0,True,Optimization terminated successfully.,"[1.7343119703913188, 1.52595574487715, 0.00105...","([[1.7343119703913188, 1.52595574487715, 0.001..."
0,6309.811035,57,156,0,True,Optimization terminated successfully.,"[0.05, 0.05755641610501444, 0.5, 0.17113604246...","([[0.05, 0.05755641610501444, 0.5, 0.171136042..."
2,6309.811035,52,160,0,True,Optimization terminated successfully.,"[0.06230622671666941, 0.05, 0.5, 0.39784159279...","([[0.06230622671666941, 0.05, 0.5, 0.397841592..."
6,6309.811035,63,173,0,True,Optimization terminated successfully.,"[0.06076885020934003, 0.05, 0.5, 0.35422315764...","([[0.06076885020934003, 0.05, 0.5, 0.354223157..."
7,6309.811035,41,134,0,True,Optimization terminated successfully.,"[0.050000000000000044, 0.06956792078475305, 0....","([[0.050000000000000044, 0.06956792078475305, ..."


#  DE fit


In [27]:
_c = next(read_from_pickle(f"ff/{ffpkl}"))

In [28]:
fit_repeat(_c, 
           NDE, 
           "test",
           bounds = DE_bound,
           loss="jax_de",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:174: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


In [29]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
7,12.099141,2212,3497,0,True,Optimization terminated successfully.,"[0.43406473249619193, 2.5, 0.12322434422358758...","([[0.43406473249619193, 2.5, 0.123224344223587..."
4,22.465603,579,1001,0,True,Optimization terminated successfully.,"[1.4906903034768606, 1.8220494210674105, 0.001...","([[1.4906903034768606, 1.8220494210674105, 0.0..."
5,29.748878,697,1168,0,True,Optimization terminated successfully.,"[2.0676642234810814, 1.6310864005739394, 0.008...","([[2.0676642234810814, 1.6310864005739394, 0.0..."
0,30.708630,875,1474,0,True,Optimization terminated successfully.,"[1.1212935286104613, 2.0048895575537795, 0.008...","([[1.1212935286104613, 2.0048895575537795, 0.0..."
2,31.311090,1150,1860,0,True,Optimization terminated successfully.,"[2.5, 1.677489497762388, 0.001000003782763078,...","([[2.5, 1.677489497762388, 0.00100000378276307..."
3,138.358719,784,1281,0,True,Optimization terminated successfully.,"[2.4608242734299077, 0.05, 0.01192326575964723...","([[2.4608242734299077, 0.05, 0.011923265759647..."
6,172.208374,608,1031,0,True,Optimization terminated successfully.,"[2.499999648123449, 0.05, 0.029897716083666502...","([[2.499999648123449, 0.05, 0.0298977160836665..."
1,6309.811035,63,238,0,True,Optimization terminated successfully.,"[0.131686342140714, 0.1623014250440108, 0.2486...","([[0.131686342140714, 0.1623014250440108, 0.24..."
8,6309.811035,36,186,0,True,Optimization terminated successfully.,"[0.3076964950101234, 0.26914206835296206, 0.47...","([[0.3076964950101234, 0.26914206835296206, 0...."
9,6309.811035,42,189,0,True,Optimization terminated successfully.,"[0.08149029599673907, 0.05987892185858845, 0.1...","([[0.08149029599673907, 0.05987892185858845, 0..."
